In [54]:
import numpy as np
import pandas as pd
import io
import requests
import json
from urllib.request import urlopen
import ipinfo
import re

In [55]:
url = "https://raw.githubusercontent.com/fspot/INFMDI-721/master/lesson5/products.csv"
df = pd.read_csv(url,sep=";")
print(type(df))
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
(200, 5)


In [56]:
#Cleaning des prix
price_sep=df["price"].str.split(" ",n=1,expand = True)
df["price"]= price_sep[0]
df["currency"]=price_sep[1]

In [57]:
df.head(15)

,username,ip_address,product,price,infos,currency
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar,None
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,None
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",None
5,bsnozzwell5,226.52.32.70,Oil - Sesame,354.33,Ingredients: sugar and milk,MGA
6,afairholme6,127.197.204.119,Chicken - Tenderloin,484.83,May contain sugar,None
7,avowdon7,189.169.17.54,Dc Hikiage Hira Huba,111.56,Contains sugar,None
8,epridham8,187.129.113.105,Dried Figs,88.05,"Ingredients: sugar, milk and fish",None
9,tkendrew9,22.32.234.215,Pop - Club Soda Can,861.25,"May contain peanut, sugar, milk and fish",None


In [58]:
df["price"]=pd.to_numeric(df.price,errors='coerce')

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 6 columns):
username      200 non-null object
ip_address    200 non-null object
product       200 non-null object
price         200 non-null float64
infos         200 non-null object
currency      17 non-null object
dtypes: float64(1), object(5)
memory usage: 9.5+ KB


In [85]:
def get_country_from_ip(ip):
    url = 'https://api.ip2country.info/ip?' + ip
    data = requests.get(url)
    try:
        output = data.json()
        return output["countryCode3"]
    except:
        return 'None'

In [86]:
ip_test = '26.191.237.49'
print(get_country_from_ip(ip_test))

USA


In [87]:
def get_currency_from_ip(ip):
    url = "https://api.ipgeolocationapi.com/geolocate/" + ip
    data = requests.get(url)
    try:
        currency = data.json()
        return currency["currency_code"]
    except:
        return "None"

def get_euro_rate(currency_code):
    if currency_code == 'None':
        return float('NaN')
    if currency_code == "EUR":
        return 1
    rate = requests.get("https://api.exchangeratesapi.io/latest?base="+currency_code)
    try:
        rate = rate.json()
        return rate["rates"]["EUR"]
    except:
        return float('NaN')

In [88]:
currency_code = [get_currency_from_ip(ip) for ip in df['ip_address']]


In [89]:
conversion_rate = [get_euro_rate(code) for code in currency_code]


In [90]:
country = [get_country_from_ip(ip) for ip in df['ip_address']]

In [91]:
df['currency_code'] = currency_code
df['conversion_rate'] = conversion_rate
df['country'] = country
df['conversion_rate'] = pd.to_numeric(df.conversion_rate,errors='coerce')
df['price_eur'] = df['price']*df['conversion_rate']

In [92]:
df.head()

,username,ip_address,product,price,infos,currency,currency_code,conversion_rate,price_eur,country
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.80,May contain sugar,None,None,NaN,NaN,None
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish,None,None,NaN,NaN,None
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish,None,None,NaN,NaN,
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten,None,USD,0.898473,314.600180,USA
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish",None,JPY,0.008273,7.857947,JPN
